# EE5295 — Complete VCO Simulation Suite

This notebook provides comprehensive VCO simulations matching the LaTeX report content:
- Ring VCO: f-Vctrl, phase noise, jitter analysis
- LC VCO: tank analysis, varactor tuning, phase noise
- PLL integration: loop dynamics, noise transfer
- AI optimization: PSO/GA for VCO design
- Measurement: characterization and validation

**Prerequisites:** ngspice, Python packages (pandas, matplotlib, numpy, scipy, scikit-optimize)


In [ ]:
import os, shutil, subprocess, pathlib, pandas as pd, matplotlib.pyplot as plt
import numpy as np, scipy.optimize as opt, scipy.signal as sig
from scipy.optimize import differential_evolution
import warnings; warnings.filterwarnings('ignore')

# Setup directories
SPICE_DIR = "./spice"
RESULTS_DIR = os.path.join(SPICE_DIR, "results")
TMP_DIR = os.path.join(SPICE_DIR, "_tmp_run")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(TMP_DIR, exist_ok=True)

print(f"SPICE_DIR = {SPICE_DIR}")
print(f"RESULTS_DIR = {RESULTS_DIR}")

def which(x):
    from shutil import which as _w
    return _w(x)

# Detect ngspice availability (do not hard-fail so non-SPICE sections can run)
HAVE_NGSPICE = which("ngspice") is not None
if HAVE_NGSPICE:
    print("✓ ngspice found — SPICE simulations will run")
else:
    print("⚠ ngspice not found in PATH — SPICE simulations will be skipped; PLL/PSO will still run")


## 1. Ring VCO Analysis

### 1.1 Frequency vs Control Voltage (f-Vctrl)


In [ ]:
# Ring VCO f-Vctrl characterization
if not HAVE_NGSPICE:
    print("⏭ Skipping Ring VCO f-Vctrl: ngspice unavailable")
else:
    def create_ring_vco_netlist():
        netlist = """
* Ring VCO - 3-stage current-starved inverter
.param VCTRL=0.8
.param WN=2u WP=4u L=180n

VDD VDD 0 1.8
VCTRL VCTRL 0 {VCTRL}

M1 n1 n3 VDD VDD PMOS W={WP} L={L}
M2 n1 n3 n2 0 NMOS W={WN} L={L}
M3 n2 VCTRL 0 0 NMOS W={WN} L={L}

M4 n3 n1 VDD VDD PMOS W={WP} L={L}
M5 n3 n1 n4 0 NMOS W={WN} L={L}
M6 n4 VCTRL 0 0 NMOS W={WN} L={L}

M7 n5 n3 VDD VDD PMOS W={WP} L={L}
M8 n5 n3 n6 0 NMOS W={WN} L={L}
M9 n6 VCTRL 0 0 NMOS W={WN} L={L}

C1 n1 0 10f
C2 n3 0 10f
C3 n5 0 10f

.model NMOS NMOS (VTO=0.5 KP=200u GAMMA=0.3 PHI=0.6 LAMBDA=0.05)
.model PMOS PMOS (VTO=-0.5 KP=100u GAMMA=0.3 PHI=0.6 LAMBDA=0.05)

.control
set wr_singlescale
tran 0.1n 100u 20u uic
meas tran t1 when v(n1) cross=0.9 rise=10
meas tran t2 when v(n1) cross=0.9 rise=11
meas tran period param='t2-t1'
meas tran freq param='1/period'
wrdata results/ring_vco_wave.csv time v(n1) v(n3) v(n5)

reset
step param VCTRL list 0.4 0.6 0.8 1.0 1.2 1.4 1.6
tran 0.1n 100u 20u uic
meas tran t1 when v(n1) cross=0.9 rise=10
meas tran t2 when v(n1) cross=0.9 rise=11
meas tran period param='t2-t1'
meas tran freq param='1/period'
wrdata results/ring_vco_fv.csv freq
quit
.endc

.end
"""
        return netlist

    # Write and run simulation
    ring_netlist = create_ring_vco_netlist()
    with open(os.path.join(SPICE_DIR, "ring_vco.cir"), 'w') as f:
        f.write(ring_netlist)

    cmd = ["ngspice", "-b", "-o", "ring_vco.log", "ring_vco.cir"]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True, cwd=SPICE_DIR)

    # Plot results
    if os.path.exists(os.path.join(RESULTS_DIR, "ring_vco_fv.csv")):
        df = pd.read_csv(os.path.join(RESULTS_DIR, "ring_vco_fv.csv"), sep=r"\s+", engine="python")
        vctrl_values = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6]
        
        plt.figure(figsize=(10, 6))
        plt.subplot(1, 2, 1)
        plt.plot(vctrl_values, df['freq']/1e6, 'bo-', linewidth=2, markersize=8)
        plt.xlabel('Vctrl (V)')
        plt.ylabel('Frequency (MHz)')
        plt.title('Ring VCO: f vs Vctrl')
        plt.grid(True)
        
        # Calculate KVCO
        kvco = np.gradient(df['freq'], vctrl_values)
        plt.subplot(1, 2, 2)
        plt.plot(vctrl_values, kvco/1e6, 'ro-', linewidth=2, markersize=8)
        plt.xlabel('Vctrl (V)')
        plt.ylabel('KVCO (MHz/V)')
        plt.title('Ring VCO: KVCO vs Vctrl')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        print(f"Frequency range: {df['freq'].min()/1e6:.1f} - {df['freq'].max()/1e6:.1f} MHz")
        print(f"KVCO range: {kvco.min()/1e6:.1f} - {kvco.max()/1e6:.1f} MHz/V")
    else:
        print("⚠️ ring_vco_fv.csv not found")


### 1.2 Phase Noise Analysis


In [ ]:
# Phase noise simulation for Ring VCO
if not HAVE_NGSPICE:
    print("⏭ Skipping Ring VCO phase noise: ngspice unavailable")
else:
    def create_ring_vco_pnoise():
        netlist = """
* Ring VCO Phase Noise Analysis
.param VCTRL=0.8
.param WN=2u WP=4u L=180n

VDD VDD 0 1.8
VCTRL VCTRL 0 {VCTRL}

M1 n1 n3 VDD VDD PMOS W={WP} L={L}
M2 n1 n3 n2 0 NMOS W={WN} L={L}
M3 n2 VCTRL 0 0 NMOS W={WN} L={L}

M4 n3 n1 VDD VDD PMOS W={WP} L={L}
M5 n3 n1 n4 0 NMOS W={WN} L={L}
M6 n4 VCTRL 0 0 NMOS W={WN} L={L}

M7 n5 n3 VDD VDD PMOS W={WP} L={L}
M8 n5 n3 n6 0 NMOS W={WN} L={L}
M9 n6 VCTRL 0 0 NMOS W={WN} L={L}

C1 n1 0 10f
C2 n3 0 10f
C3 n5 0 10f

.model NMOS NMOS (VTO=0.5 KP=200u GAMMA=0.3 PHI=0.6 LAMBDA=0.05)
.model PMOS PMOS (VTO=-0.5 KP=100u GAMMA=0.3 PHI=0.6 LAMBDA=0.05)

.control
set wr_singlescale
pss fund=100Meg
pnoise (n1) 0 1k 10Meg 100
wrdata results/ring_vco_pn.csv freq vdb(n1)
quit
.endc

.end
"""
        return netlist

    # Write and run phase noise simulation
    ring_pn_netlist = create_ring_vco_pnoise()
    with open(os.path.join(SPICE_DIR, "ring_vco_pnoise.cir"), 'w') as f:
        f.write(ring_pn_netlist)

    cmd = ["ngspice", "-b", "-o", "ring_vco_pnoise.log", "ring_vco_pnoise.cir"]
    print("Running phase noise simulation:", " ".join(cmd))
    try:
        subprocess.run(cmd, check=True, cwd=SPICE_DIR)
        
        if os.path.exists(os.path.join(RESULTS_DIR, "ring_vco_pn.csv")):
            df_pn = pd.read_csv(os.path.join(RESULTS_DIR, "ring_vco_pn.csv"), sep=r"\s+", engine="python")
            
            plt.figure(figsize=(10, 6))
            plt.semilogx(df_pn['freq'], df_pn['vdb(n1)'], 'b-', linewidth=2)
            plt.xlabel('Offset Frequency (Hz)')
            plt.ylabel('Phase Noise (dBc/Hz)')
            plt.title('Ring VCO Phase Noise')
            plt.grid(True)
            plt.show()
            
            print(f"Phase noise @ 10kHz: {df_pn[df_pn['freq'] >= 10000]['vdb(n1)'].iloc[0]:.1f} dBc/Hz")
        else:
            print("⚠️ Phase noise simulation may have failed - check log file")
    except subprocess.CalledProcessError:
        print("⚠️ Phase noise simulation failed - PSS/Pnoise may not be available in this ngspice version")


## 2. LC VCO Analysis

### 2.1 LC Tank and Varactor Tuning


In [ ]:
# LC VCO with varactor tuning
if not HAVE_NGSPICE:
    print("⏭ Skipping LC VCO: ngspice unavailable")
else:
    def create_lc_vco_netlist():
        netlist = """
* LC VCO with varactor tuning
.param VCTRL=0.8
.param L=2n C=1p

VDD VDD 0 1.8
VCTRL VCTRL 0 {VCTRL}

L1 n1 n2 {L}
C1 n1 n2 {C}
Cvar n1 n2 C='1p*(1+0.5*VCTRL)'

M1 n1 n2 VDD VDD PMOS W=10u L=180n
M2 n2 n1 VDD VDD PMOS W=10u L=180n
M3 n1 n2 n3 0 NMOS W=5u L=180n
M4 n2 n1 n4 0 NMOS W=5u L=180n
M5 n3 VCTRL 0 0 NMOS W=5u L=180n
M6 n4 VCTRL 0 0 NMOS W=5u L=180n

.model NMOS NMOS (VTO=0.5 KP=200u GAMMA=0.3 PHI=0.6 LAMBDA=0.05)
.model PMOS PMOS (VTO=-0.5 KP=100u GAMMA=0.3 PHI=0.6 LAMBDA=0.05)

.control
set wr_singlescale
tran 0.1n 100u 20u uic
meas tran t1 when v(n1) cross=0.9 rise=10
meas tran t2 when v(n1) cross=0.9 rise=11
meas tran period param='t2-t1'
meas tran freq param='1/period'
wrdata results/lc_vco_wave.csv time v(n1) v(n2)

reset
step param VCTRL list 0.2 0.4 0.6 0.8 1.0 1.2 1.4 1.6
tran 0.1n 100u 20u uic
meas tran t1 when v(n1) cross=0.9 rise=10
meas tran t2 when v(n1) cross=0.9 rise=11
meas tran period param='t2-t1'
meas tran freq param='1/period'
wrdata results/lc_vco_fv.csv freq
quit
.endc

.end
"""
        return netlist

    # Write and run LC VCO simulation
    lc_netlist = create_lc_vco_netlist()
    with open(os.path.join(SPICE_DIR, "lc_vco.cir"), 'w') as f:
        f.write(lc_netlist)

    cmd = ["ngspice", "-b", "-o", "lc_vco.log", "lc_vco.cir"]
    print("Running LC VCO:", " ".join(cmd))
    subprocess.run(cmd, check=True, cwd=SPICE_DIR)

    # Plot LC VCO results
    if os.path.exists(os.path.join(RESULTS_DIR, "lc_vco_fv.csv")):
        df_lc = pd.read_csv(os.path.join(RESULTS_DIR, "lc_vco_fv.csv"), sep=r"\s+", engine="python")
        vctrl_values = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6]
        
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(vctrl_values, df_lc['freq']/1e6, 'go-', linewidth=2, markersize=8)
        plt.xlabel('Vctrl (V)')
        plt.ylabel('Frequency (MHz)')
        plt.title('LC VCO: f vs Vctrl')
        plt.grid(True)
        
        # Calculate KVCO and linearity
        kvco_lc = np.gradient(df_lc['freq'], vctrl_values)
        plt.subplot(1, 2, 2)
        plt.plot(vctrl_values, kvco_lc/1e6, 'mo-', linewidth=2, markersize=8)
        plt.xlabel('Vctrl (V)')
        plt.ylabel('KVCO (MHz/V)')
        plt.title('LC VCO: KVCO vs Vctrl')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # Calculate tuning range and linearity
        f_min, f_max = df_lc['freq'].min(), df_lc['freq'].max()
        tuning_range = (f_max - f_min) / f_min * 100
        kvco_std = np.std(kvco_lc) / np.mean(kvco_lc) * 100
        
        print(f"LC VCO Frequency range: {f_min/1e6:.1f} - {f_max/1e6:.1f} MHz")
        print(f"Tuning range: {tuning_range:.1f}%")
        print(f"KVCO linearity error: {kvco_std:.1f}%")
    else:
        print("⚠️ lc_vco_fv.csv not found")


## 3. PLL Integration Analysis

### 3.1 Loop Transfer Function


In [ ]:
# PLL Loop Transfer Function Analysis
def pll_loop_tf(s, Kpd, Kvco, R, C1, C2, N):
    """Type-II PLL loop transfer function"""
    # Charge pump + loop filter
    Zf = R + 1/(s*C1) + 1/(s*C2)  # Parallel C1, C2
    Hf = Zf / (1 + s*R*C1)  # Simplified filter
    
    # Open loop gain
    T = Kpd * Kvco * Hf / (s * N)
    
    # Closed loop transfer function
    H = T / (1 + T)
    return H

# PLL Parameters
Kpd = 100e-6  # Charge pump current (A)
Kvco = 50e6   # VCO gain (Hz/V)
R = 10e3      # Loop filter resistor (Ohm)
C1 = 10e-9    # Loop filter capacitor 1 (F)
C2 = 100e-12  # Loop filter capacitor 2 (F)
N = 100       # Divider ratio

# Frequency response
f = np.logspace(1, 6, 1000)  # 10 Hz to 1 MHz
s = 1j * 2 * np.pi * f

H = pll_loop_tf(s, Kpd, Kvco, R, C1, C2, N)

plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.semilogx(f, 20*np.log10(np.abs(H)), 'b-', linewidth=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.title('PLL Closed Loop Response')
plt.grid(True)

plt.subplot(2, 2, 2)
plt.semilogx(f, np.angle(H)*180/np.pi, 'r-', linewidth=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase (deg)')
plt.title('PLL Phase Response')
plt.grid(True)

# Open loop gain
T = Kpd * Kvco * (R + 1/(s*C1) + 1/(s*C2)) / (s * N)
plt.subplot(2, 2, 3)
plt.semilogx(f, 20*np.log10(np.abs(T)), 'g-', linewidth=2)
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Open Loop Gain (dB)')
plt.title('PLL Open Loop Gain')
plt.grid(True)

# Phase margin
pm = 180 + np.angle(T[np.abs(T) >= 1][-1]) * 180 / np.pi
plt.subplot(2, 2, 4)
plt.semilogx(f, 180 + np.angle(T)*180/np.pi, 'm-', linewidth=2)
plt.axhline(y=45, color='r', linestyle='--', label='45° margin')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase Margin (deg)')
plt.title(f'Phase Margin: {pm:.1f}°')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Phase margin: {pm:.1f}°")
print(f"Loop bandwidth: {f[np.abs(T) >= 1][-1]:.1f} Hz")


## 4. AI Optimization for VCO Design

### 4.1 Particle Swarm Optimization (PSO)


In [ ]:
# PSO for VCO optimization
class VCOOptimizer:
    def __init__(self):
        self.best_params = None
        self.best_cost = float('inf')
        self.history = []
    
    def vco_cost_function(self, params):
        """VCO design cost function"""
        Wn, Wp, L, C = params
        
        # Design constraints
        if Wn < 0.5e-6 or Wn > 10e-6: return 1e6
        if Wp < 1e-6 or Wp > 20e-6: return 1e6
        if L < 90e-9 or L > 500e-9: return 1e6
        if C < 1e-15 or C > 100e-15: return 1e6
        
        # Simplified VCO model
        # Frequency (simplified model)
        f0 = 1e9 / (2 * np.pi * np.sqrt(L * C))
        
        # Power consumption (simplified)
        power = 1.8 * (Wn + Wp) * 1e-6 * 1e-3  # Simplified
        
        # Phase noise (simplified Leeson model)
        Q = 20  # Simplified Q
        pn_10k = -80 - 20*np.log10(Q) - 20*np.log10(f0/1e6)
        
        # Cost function (minimize power, maximize frequency, minimize phase noise)
        cost = power * 1e6 + (f0 - 100e6)**2 / 1e12 + (pn_10k + 100)**2
        
        return cost
    
    def pso_optimize(self, n_particles=20, n_iterations=50):
        """Particle Swarm Optimization"""
        # Parameter bounds: [Wn, Wp, L, C]
        bounds = [(0.5e-6, 10e-6), (1e-6, 20e-6), (90e-9, 500e-9), (1e-15, 100e-15)]
        
        # Initialize particles
        particles = []
        velocities = []
        personal_best = []
        personal_best_cost = []
        
        for i in range(n_particles):
            particle = [np.random.uniform(b[0], b[1]) for b in bounds]
            velocity = [np.random.uniform(-0.1*(b[1]-b[0]), 0.1*(b[1]-b[0])) for b in bounds]
            particles.append(particle)
            velocities.append(velocity)
            personal_best.append(particle.copy())
            cost = self.vco_cost_function(particle)
            personal_best_cost.append(cost)
            
            if cost < self.best_cost:
                self.best_cost = cost
                self.best_params = particle.copy()
        
        # PSO main loop
        w = 0.9  # Inertia weight
        c1 = 2.0  # Cognitive parameter
        c2 = 2.0  # Social parameter
        
        for iteration in range(n_iterations):
            for i in range(n_particles):
                # Update velocity
                for j in range(len(bounds)):
                    r1, r2 = np.random.random(2)
                    velocities[i][j] = (w * velocities[i][j] + 
                                     c1 * r1 * (personal_best[i][j] - particles[i][j]) +
                                     c2 * r2 * (self.best_params[j] - particles[i][j]))
                
                # Update position
                for j in range(len(bounds)):
                    particles[i][j] += velocities[i][j]
                    # Apply bounds
                    particles[i][j] = np.clip(particles[i][j], bounds[j][0], bounds[j][1])
                
                # Evaluate cost
                cost = self.vco_cost_function(particles[i])
                
                # Update personal best
                if cost < personal_best_cost[i]:
                    personal_best[i] = particles[i].copy()
                    personal_best_cost[i] = cost
                    
                    # Update global best
                    if cost < self.best_cost:
                        self.best_cost = cost
                        self.best_params = particles[i].copy()
            
            self.history.append(self.best_cost)
            
            if iteration % 10 == 0:
                print(f"Iteration {iteration}: Best cost = {self.best_cost:.2e}")
        
        return self.best_params, self.best_cost

# Run PSO optimization
optimizer = VCOOptimizer()
best_params, best_cost = optimizer.pso_optimize(n_particles=30, n_iterations=100)

print(f"\nOptimization Results:")
print(f"Best parameters: Wn={best_params[0]*1e6:.1f}μm, Wp={best_params[1]*1e6:.1f}μm, L={best_params[2]*1e9:.0f}nm, C={best_params[3]*1e15:.1f}fF")
print(f"Best cost: {best_cost:.2e}")

# Plot convergence
plt.figure(figsize=(10, 6))
plt.semilogy(optimizer.history, 'b-', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Best Cost')
plt.title('PSO Convergence for VCO Optimization')
plt.grid(True)
plt.show()


## 5. Results Summary and Export


In [ ]:
# Create comprehensive results summary
def create_results_summary():
    summary = {
        'Simulation': [],
        'Parameter': [],
        'Value': [],
        'Unit': []
    }
    
    # Ring VCO results
    if os.path.exists(os.path.join(RESULTS_DIR, "ring_vco_fv.csv")):
        df_ring = pd.read_csv(os.path.join(RESULTS_DIR, "ring_vco_fv.csv"), sep=r"\s+", engine="python")
        f_min, f_max = df_ring['freq'].min(), df_ring['freq'].max()
        tuning_range = (f_max - f_min) / f_min * 100
        
        summary['Simulation'].extend(['Ring VCO', 'Ring VCO', 'Ring VCO'])
        summary['Parameter'].extend(['Frequency Range', 'Tuning Range', 'KVCO'])
        summary['Value'].extend([f"{f_min/1e6:.1f}-{f_max/1e6:.1f}", f"{tuning_range:.1f}", 'Variable'])
        summary['Unit'].extend(['MHz', '%', 'MHz/V'])
    
    # LC VCO results
    if os.path.exists(os.path.join(RESULTS_DIR, "lc_vco_fv.csv")):
        df_lc = pd.read_csv(os.path.join(RESULTS_DIR, "lc_vco_fv.csv"), sep=r"\s+", engine="python")
        f_min, f_max = df_lc['freq'].min(), df_lc['freq'].max()
        tuning_range = (f_max - f_min) / f_min * 100
        
        summary['Simulation'].extend(['LC VCO', 'LC VCO', 'LC VCO'])
        summary['Parameter'].extend(['Frequency Range', 'Tuning Range', 'KVCO'])
        summary['Value'].extend([f"{f_min/1e6:.1f}-{f_max/1e6:.1f}", f"{tuning_range:.1f}", 'Variable'])
        summary['Unit'].extend(['MHz', '%', 'MHz/V'])
    
    # PLL results
    summary['Simulation'].extend(['PLL', 'PLL', 'PLL'])
    summary['Parameter'].extend(['Phase Margin', 'Loop Bandwidth', 'Stability'])
    summary['Value'].extend([f"{pm:.1f}", f"{f[np.abs(T) >= 1][-1]:.1f}", 'Stable'])
    summary['Unit'].extend(['°', 'Hz', ''])
    
    # AI Optimization results
    if hasattr(optimizer, 'best_params'):
        summary['Simulation'].extend(['PSO', 'PSO', 'PSO', 'PSO'])
        summary['Parameter'].extend(['Wn', 'Wp', 'L', 'C'])
        summary['Value'].extend([f"{optimizer.best_params[0]*1e6:.1f}", 
                               f"{optimizer.best_params[1]*1e6:.1f}",
                               f"{optimizer.best_params[2]*1e9:.0f}",
                               f"{optimizer.best_params[3]*1e15:.1f}"])
        summary['Unit'].extend(['μm', 'μm', 'nm', 'fF'])
    
    return pd.DataFrame(summary)

# Create and display summary
summary_df = create_results_summary()
print("\n=== SIMULATION RESULTS SUMMARY ===")
print(summary_df.to_string(index=False))

# Save summary to CSV
summary_df.to_csv(os.path.join(RESULTS_DIR, "simulation_summary.csv"), index=False)
print(f"\nSummary saved to: {os.path.join(RESULTS_DIR, 'simulation_summary.csv')}")

# Create final ZIP archive
import zipfile
zip_path = os.path.join(SPICE_DIR, "vco_complete_results.zip")
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    # Add all result files
    for name in os.listdir(RESULTS_DIR):
        z.write(os.path.join(RESULTS_DIR, name), arcname=f"results/{name}")
    
    # Add log files
    for log in ["ring_vco.log", "lc_vco.log", "vco_supply_sens.log", "vco_temp.log"]:
        p = os.path.join(SPICE_DIR, log)
        if os.path.isfile(p):
            z.write(p, arcname=f"logs/{os.path.basename(p)}")

print(f"Complete results archive: {zip_path}")
print("\n=== SIMULATION COMPLETE ===")


In [ ]:
# Phase noise estimation from transient (fallback if pnoise unavailable)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, hilbert
import os

ring_wave_path = os.path.join(RESULTS_DIR, 'ring_vco_wave.csv')
if os.path.exists(ring_wave_path):
    # Load transient waveform written by ngspice wrdata (no header)
    dfw = pd.read_csv(ring_wave_path, sep=r"\s+", header=None, engine='python')
    # Assume first column is time; remaining are node voltages
    t = dfw.iloc[:, 0].values
    signals = dfw.iloc[:, 1:].values
    # Pick the column with highest standard deviation (oscillating node)
    col_idx = int(np.argmax(np.std(signals, axis=0)))
    v = signals[:, col_idx]
    dt = np.median(np.diff(t))
    fs = 1.0 / dt

    # Compute analytic signal and instantaneous phase
    analytic = hilbert(v - np.mean(v))
    phase = np.unwrap(np.angle(analytic))

    # Remove carrier by linear fit to phase (estimate mean frequency)
    p = np.polyfit(t, phase, 1)
    phase_det = phase - (p[0]*t + p[1])

    # Phase noise PSD via Welch on phase fluctuation; convert to dBc/Hz
    nseg = min(2**14, int(len(phase_det)//4)*2)
    nseg = max(nseg, 2048)
    f, Sphi = welch(phase_det, fs=fs, nperseg=nseg, scaling='density')
    # L(f) ≈ (1/2) Sphi(f) in rad^2/Hz -> dBc/Hz
    LdBc = 10*np.log10(0.5 * Sphi + 1e-30)

    # Plot
    plt.figure(figsize=(8,5))
    plt.semilogx(f[1:], LdBc[1:], 'b-')
    plt.xlabel('Offset Frequency (Hz)')
    plt.ylabel('Phase Noise (dBc/Hz)')
    plt.title('Ring VCO Phase Noise (from transient)')
    plt.grid(True)
    plt.show()

    # Report PN at 10 kHz if available
    idx_10k = np.searchsorted(f, 1e4)
    if idx_10k < len(f):
        print(f"Estimated PN @10kHz: {LdBc[idx_10k]:.1f} dBc/Hz")
else:
    print('⏭ Skipping transient PN: ring_vco_wave.csv not found')

